In [1]:
# 1. 환경 확인
import torch, transformers, bitsandbytes
print("torch:", torch.__version__)
print("transformers:", transformers.__version__)

# 2. extractor_model import
from extractor_model import extract_keywords

/home/alpaco/anaconda3/envs/fridge/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


torch: 2.9.1+cu128
transformers: 4.57.1
Loading tokenizer...
Loading model (this can take a while)...


Loading checkpoint shards: 100%|██████████| 8/8 [00:23<00:00,  2.99s/it]


In [3]:
from collections import Counter

# Qwen 모델 로딩이 끝났다고 가정 (extractor_model.py에서 model 변수 사용 중)
from extractor_model import model

# 1) hf_device_map 있는지 확인
print("has hf_device_map:", hasattr(model, "hf_device_map"))

if hasattr(model, "hf_device_map"):
    dm = model.hf_device_map
    print("Total modules in device_map:", len(dm))
    print("Sample entries:")
    for name, dev in list(dm.items())[:20]:
        print(f"  {name:40s} -> {dev}")

    # 2) 각 디바이스에 몇 개의 모듈이 올라갔는지 요약
    counts = Counter(dm.values())
    print("\nModule count per device:")
    for dev, cnt in counts.items():
        print(f"  {dev}: {cnt}")

has hf_device_map: True
Total modules in device_map: 52
Sample entries:
  model.embed_tokens                       -> 0
  model.layers.0                           -> 0
  model.layers.1                           -> 0
  model.layers.2                           -> 0
  model.layers.3                           -> 0
  model.layers.4                           -> 0
  model.layers.5                           -> 1
  model.layers.6                           -> 1
  model.layers.7                           -> 1
  model.layers.8                           -> 1
  model.layers.9                           -> 1
  model.layers.10                          -> 1
  model.layers.11                          -> 1
  model.layers.12                          -> 1
  model.layers.13                          -> 1
  model.layers.14                          -> 1
  model.layers.15                          -> 1
  model.layers.16                          -> 1
  model.layers.17                          -> 1
  model.layers.1

In [4]:
import torch

for i in range(torch.cuda.device_count()):
    mem_gb = torch.cuda.memory_allocated(i) / (1024 ** 3)
    print(f"GPU {i} memory allocated: {mem_gb:.2f} GB")

GPU 0 memory allocated: 2.12 GB
GPU 1 memory allocated: 2.53 GB
GPU 2 memory allocated: 4.64 GB


In [3]:
test_prompts = [
    # 1. 기본 국물 + 재료 + 안 매운
    "따뜻하게 먹을 수 있는 국물요리 추천해줘. 냉장고에 떡, 계란이 있고 너무 매콤하지 않았으면 좋겠어.",

    # 2. 돼지고기 제외 + 채식 위주
    "국이나 탕 느낌으로 담백한 한식 먹고 싶어. 돼지고기는 싫고, 소고기도 좀 피하고 싶어. 채식 위주로 먹고 있어.",

    # 3. 특정 재료 제외 + 시간 조건
    "30분 안에 만들 수 있는 간단한 볶음 요리 추천해줘. 마늘은 알레르기가 있어서 빼줘.",

    # 4. 비건
    "나는 비건이야. 동물성 재료 하나도 안 들어간 면요리나 국물요리 있으면 추천해줘.",

    # 5. 술안주 + 매운거 OK + 상황
    "친구들이랑 집에서 술 마실건데, 술안주로 먹기 좋은 약간 매콤한 요리 추천해줘. 튀김 말고 다른 걸로."
]

In [4]:
import pprint
pp = pprint.PrettyPrinter(indent=2, width=120)

for i, prompt in enumerate(test_prompts, start=1):
    print("=" * 80)
    print(f"[Test {i}] User prompt:")
    print(prompt)
    print("\n[Extracted keywords JSON]:")
    kw = extract_keywords(prompt)
    pp.pprint(kw)
    print("\n")

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


[Test 1] User prompt:
따뜻하게 먹을 수 있는 국물요리 추천해줘. 냉장고에 떡, 계란이 있고 너무 매콤하지 않았으면 좋겠어.

[Extracted keywords JSON]:
{ 'dish_type': ['국물요리'],
  'free_text': '따뜻하게 먹을 수 있는 국물요리를 추천해주세요. 냉장고에 떡과 계란이 있으며, 매운 정도가 너무 매콤하지 않게 해주세요.',
  'must_ingredients': ['떡', '계란'],
  'situation': ['따뜻하게'],
  'spiciness': 'low'}


[Test 2] User prompt:
국이나 탕 느낌으로 담백한 한식 먹고 싶어. 돼지고기는 싫고, 소고기도 좀 피하고 싶어. 채식 위주로 먹고 있어.

[Extracted keywords JSON]:
{ 'dietary_constraints': { 'no_beef': True,
                           'no_chicken': False,
                           'no_pork': True,
                           'no_seafood': False,
                           'vegan': False,
                           'vegetarian': True},
  'dish_type': ['국', '탕'],
  'exclude_ingredients': ['돼지고기', '소고기'],
  'free_text': '담백한 국이나 탕을 원하며, 돼지고기와 소고기를 피하고 싶습니다. 채식주의자입니다.',
  'situation': ['담백한 한식']}


[Test 3] User prompt:
30분 안에 만들 수 있는 간단한 볶음 요리 추천해줘. 마늘은 알레르기가 있어서 빼줘.

[Extracted keywords JSON]:
{ 'difficulty': ['간단'],
  'dish_type': ['볶음'],


In [5]:
kw

{'dish_type': ['술안주'],
 'method': [],
 'situation': ['집에서'],
 'must_ingredients': [],
 'optional_ingredients': [],
 'exclude_ingredients': ['튀김'],
 'spiciness': 'low',
 'dietary_constraints': {'vegetarian': False,
  'vegan': False,
  'no_beef': False,
  'no_pork': False,
  'no_chicken': False,
  'no_seafood': False},
 'servings': {'min': None, 'max': None},
 'max_cook_time_min': None,
 'difficulty': [],
 'positive_tags': [],
 'negative_tags': ['튀김'],
 'free_text': '매콤한 술안주를 찾고 계시며, 튀김은 제외하고 싶으신 것 같습니다.'}

In [2]:
from neo4j import GraphDatabase

URI = "bolt://localhost:7687"
USER = "neo4j"
PASSWORD = "password"

driver = GraphDatabase.driver(URI, auth=(USER, PASSWORD))

In [3]:
def build_cypher_from_keywords(kw: dict, limit: int = 50):
    where_clauses = []
    params = {}

    # 1) must_ingredients: 반드시 포함해야 하는 재료
    if kw.get("must_ingredients"):
        where_clauses.append("""
        ALL(ing IN $must_ings WHERE
            EXISTS {
                MATCH (r)-[:HAS_INGREDIENT]->(:Ingredient {name: ing})
            }
        )
        """)
        params["must_ings"] = kw["must_ingredients"]

    # 2) exclude_ingredients: 포함되면 안 되는 재료
    if kw.get("exclude_ingredients"):
        where_clauses.append("""
        ALL(ing IN $exclude_ings WHERE
            NOT EXISTS {
                MATCH (r)-[:HAS_INGREDIENT]->(:Ingredient {name: ing})
            }
        )
        """)
        params["exclude_ings"] = kw["exclude_ingredients"]

    # 3) dish_type -> Category (예: 국, 탕, 볶음)
    if kw.get("dish_type"):
        where_clauses.append("""
        EXISTS {
            MATCH (r)-[:IN_CATEGORY]->(c:Category)
            WHERE c.name IN $cats
        }
        """)
        params["cats"] = kw["dish_type"]

    # 4) method -> Method (예: 끓이기, 볶기)
    if kw.get("method"):
        where_clauses.append("""
        EXISTS {
            MATCH (r)-[:COOKED_BY]->(m:Method)
            WHERE m.name IN $methods
        }
        """)
        params["methods"] = kw["method"]

    # 5) situation -> Situation (예: 명절, 야식)
    if kw.get("situation"):
        where_clauses.append("""
        EXISTS {
            MATCH (r)-[:FOR_SITUATION]->(s:Situation)
            WHERE s.name IN $sits
        }
        """)
        params["sits"] = kw["situation"]

    # 6) 난이도
    if kw.get("difficulty"):
        where_clauses.append("r.difficulty IN $diffs")
        params["diffs"] = kw["difficulty"]

    # 7) 최대 조리 시간
    if kw.get("max_cook_time_min"):
        where_clauses.append("r.time_min IS NOT NULL AND r.time_min <= $tmax")
        params["tmax"] = kw["max_cook_time_min"]

    where_str = ""
    if where_clauses:
        where_str = "WHERE " + " AND ".join([w.strip() for w in where_clauses])

    cypher = f"""
    MATCH (r:Recipe)
    {where_str}
    RETURN
        r.recipe_id AS recipe_id,
        r.title AS title,
        r.name AS name,
        r.views AS views,
        r.time_min AS time_min,
        r.difficulty AS difficulty,
        r.servings AS servings
    ORDER BY r.views DESC
    LIMIT {limit}
    """

    return cypher, params

In [4]:
def build_cypher_from_keywords_relaxed(kw: dict, limit: int = 50):
    where_clauses = []
    params = {}

    # 1) must_ingredients: "이 문자열이 이름에 포함되는 재료" 있으면 OK
    ing_filters = []
    if kw.get("must_ingredients"):
        for idx, ing in enumerate(kw["must_ingredients"]):
            key = f"must_ing_{idx}"
            # LOWER 비교도 가능
            clause = f"""
            EXISTS {{
                MATCH (r)-[:HAS_INGREDIENT]->(mi:Ingredient)
                WHERE mi.name CONTAINS ${key}
            }}
            """
            ing_filters.append(clause)
            params[key] = ing  # 예: "떡", "계란"
    if ing_filters:
        where_clauses.append("(" + " AND ".join(ing_filters) + ")")

    # 2) exclude_ingredients: 이름에 포함되면 안 됨
    exc_filters = []
    if kw.get("exclude_ingredients"):
        for idx, ing in enumerate(kw["exclude_ingredients"]):
            key = f"exc_ing_{idx}"
            clause = f"""
            NOT EXISTS {{
                MATCH (r)-[:HAS_INGREDIENT]->(ei:Ingredient)
                WHERE ei.name CONTAINS ${key}
            }}
            """
            exc_filters.append(clause)
            params[key] = ing
    if exc_filters:
        where_clauses.append("(" + " AND ".join(exc_filters) + ")")

    # 3) dish_type은 일단 잠깐 빼거나, 간단 매핑으로 변환
    # 예: "국물요리" -> ["국", "탕", "찌개"]
    # 여기서는 일단 skip (나중에 mapping table로 개선)

    # 4) method/situation도 일단 빼고 재료 기반으로만 필터
    # 나중에 mapping 잘 설계하고 추가

    # 5) 시간 조건 정도는 유지해도 괜찮음
    if kw.get("max_cook_time_min"):
        where_clauses.append("r.time_min IS NOT NULL AND r.time_min <= $tmax")
        params["tmax"] = kw["max_cook_time_min"]

    where_str = ""
    if where_clauses:
        where_str = "WHERE " + " AND ".join([w.strip() for w in where_clauses])

    cypher = f"""
    MATCH (r:Recipe)
    {where_str}
    RETURN
        r.recipe_id AS recipe_id,
        r.title AS title,
        r.name AS name,
        r.views AS views,
        r.time_min AS time_min,
        r.difficulty AS difficulty,
        r.servings AS servings
    ORDER BY r.views DESC
    LIMIT {limit}
    """

    return cypher, params

In [5]:
def graph_rag_search(user_prompt: str, top_k: int = 10):
    # 1) 키워드 추출
    kw = extract_keywords(user_prompt)
    print("=== Extracted keywords ===")
    from pprint import pprint
    pprint(kw)

    # 2) Cypher 쿼리 생성
    cypher, params = build_cypher_from_keywords_relaxed(kw, limit=50)
    print("\n=== Generated Cypher ===")
    print(cypher)
    print("\nParams:", params)

    # 3) Neo4j에서 후보 레시피 검색
    with driver.session() as session:
        result = session.run(cypher, **params)
        rows = list(result)

    # 4) 상위 top_k개만 잘라서 보기 좋게 포맷
    recipes = []
    for rec in rows[:top_k]:
        recipes.append({
            "recipe_id": rec["recipe_id"],
            "title": rec["title"],
            "name": rec["name"],
            "views": rec["views"],
            "time_min": rec["time_min"],
            "difficulty": rec["difficulty"],
            "servings": rec["servings"],
        })

    print(f"\n=== Top {top_k} results ===")
    for i, r in enumerate(recipes, start=1):
        print(f"[{i}] ({r['recipe_id']}) {r['title']} | {r['time_min']}분 | {r['difficulty']} | 조회수 {r['views']}")

    return {
        "keywords": kw,
        "recipes": recipes,
    }

In [6]:
queries = [
    "따뜻하게 먹을 수 있는 국물요리 추천해줘. 냉장고에 떡, 계란이 있고 너무 매콤하지 않았으면 좋겠어.",
    # "돼지고기는 싫고, 소고기도 안 먹어. 채식 위주로 먹고 있는데 담백한 국이나 찌개 추천해줘.",
    # "30분 안에 만들 수 있는 간단한 볶음요리 추천해줘. 마늘은 알레르기라 빼줘.",
]

for q in queries:
    print("\n" + "="*80)
    print("USER PROMPT:", q)
    res = graph_rag_search(q, top_k=50)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



USER PROMPT: 따뜻하게 먹을 수 있는 국물요리 추천해줘. 냉장고에 떡, 계란이 있고 너무 매콤하지 않았으면 좋겠어.
=== Extracted keywords ===
{'dish_type': ['국물요리'],
 'free_text': '따뜻하게 먹을 수 있는 국물요리를 추천해주세요. 냉장고에 떡과 계란이 있으며 매운 정도가 너무 매콤하지 않게 '
              '해주세요.',
 'must_ingredients': ['떡', '계란'],
 'situation': ['따뜻하게'],
 'spiciness': 'low'}

=== Generated Cypher ===

    MATCH (r:Recipe)
    WHERE (
            EXISTS {
                MATCH (r)-[:HAS_INGREDIENT]->(mi:Ingredient)
                WHERE mi.name CONTAINS $must_ing_0
            }
             AND 
            EXISTS {
                MATCH (r)-[:HAS_INGREDIENT]->(mi:Ingredient)
                WHERE mi.name CONTAINS $must_ing_1
            }
            )
    RETURN
        r.recipe_id AS recipe_id,
        r.title AS title,
        r.name AS name,
        r.views AS views,
        r.time_min AS time_min,
        r.difficulty AS difficulty,
        r.servings AS servings
    ORDER BY r.views DESC
    LIMIT 50
    

Params: {'must_ing_0': '떡', 'must_ing_1': '계란'}

In [13]:
res

{'keywords': {'dish_type': ['국물요리'],
  'situation': ['따뜻하게'],
  'must_ingredients': ['떡', '계란'],
  'spiciness': 'low',
  'free_text': '따뜻하게 먹을 수 있는 국물요리를 추천해주세요. 냉장고에 떡과 계란이 있으며, 매운 정도가 너무 매콤하지 않게 해주세요.'},
 'recipes': []}